In [10]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [12]:
def scrap_nike_reviews(business_url):
    response = requests.get(business_url)

    # Vérifiez si la requête a réussi
    if response.status_code != 200:
        print(f"Erreur lors de la récupération des données : {response.status_code}")
        return

    # Analysez le contenu avec BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extraction des commentaires
    comments = [review.get_text(strip=True) for review in soup.find_all('div', class_="styles_reviewContent__0Q2Tg")]

    # Extraction des notes
    notes = [
        rating.find('img')['alt'].split(' ')[1]
        for rating in soup.find_all('div', class_="star-rating_starRating__4rrcf star-rating_medium__iN6Ty")
        if rating.find('img') and 'Noté' in rating.find('img')['alt']
    ]

    # Extraction des pays
    countries_list = [
        country.find('span').get_text(strip=True)
        for country in soup.find_all('div', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua')
    ]

    # Extraction du nombre de notations
    numer_ratings_list = [
        number_rating.get_text(strip=True)
        for number_rating in soup.find_all('span', class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l")
    ]

    return comments, notes, countries_list, numer_ratings_list[3:]


In [13]:
def data_to_dataframe():
    all_reviews = []
    all_notes = []
    all_countries = []
    all_number_ratings = []

    # Itérer sur plusieurs pages
    for page in range(1, 42):
        business_url = f'https://fr.trustpilot.com/review/www.nike.com?page={page}'
        reviews, notes, countries, number_ratings = scrap_nike_reviews(business_url)
        
        # Ajouter les données de cette page à la liste globale
        all_reviews.extend(reviews)
        all_notes.extend(notes)
        all_countries.extend(countries)
        all_number_ratings.extend(number_ratings)
    
    # Créer un DataFrame avec les listes collectées
    df = pd.DataFrame({
        'Review': all_reviews,
        'Rating': all_notes,
        'Country': all_countries,
        'user_avis': all_number_ratings
    })
    
    return df

In [14]:
df = data_to_dataframe()

In [15]:
df['Date'] = df['Review'].apply(lambda x : x[x.find("Date de l'expérience") + len("Date de l'expérience") + 1:])
df['Review'] = df['Review'].apply(lambda x : x[:x.find("Date de l'expérience")])

In [16]:
df

,Review,Rating,Country,user_avis,Date
0,Pas de commande sans compteImpossible de passe...,1,FR,1avis,11 juillet 2024
1,Ne jamais commander sur NikeNe jamais commande...,1,FR,1avis,20 juin 2024
2,Jamais déçu du process de commandeJamais déçu ...,5,FR,2avis,24 octobre 2024
3,2 paires de TN full cuir défectueuse2 paires d...,1,BE,12avis,16 octobre 2024
4,N'achetez pas NikeLe service client que ce soi...,1,FR,6avis,01 août 2024
...,...,...,...,...,...
806,Excellent customer service and shop staffCusto...,5,FR,3avis,31 juillet 2016
807,Commande reçue très rapidementCommandé sur le ...,5,FR,13avis,26 mars 2016
808,Des articles personnalisésQuand je peux me le ...,4,NL,9avis,02 juillet 2015
809,parfaittout est très bien.on se ballade facile...,5,FR,3avis,21 novembre 2014


In [9]:
df.to_csv('avis_nike.csv', index=False)